Calculate the Inception Score (IS) and Frechet Inception Distance (FID) between real images and themselves as a reference.

In [3]:
import torch
from torchvision import models, transforms
from torchvision.models import Inception_V3_Weights
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inception v3 
inception = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
inception = inception.to(device).eval()
inception.fc = torch.nn.Sequential(inception.fc, torch.nn.Softmax(dim=1))

preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_images_from_folder(folder, preprocess):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        with Image.open(img_path) as img:
            img = preprocess(img)
            images.append(img)
    return torch.stack(images)

def get_activations(images, model, batch_size=64):
    model.eval()
    n_batches = len(images) // batch_size
    n_used_imgs = n_batches * batch_size
    pred_arr = np.empty((n_used_imgs, 1000))
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        batch = images[start:end]
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch) 
        pred_arr[start:end] = pred.cpu().data.numpy()
    return pred_arr

def calculate_inception_score(p_yx, eps=1E-16):
    p_y = np.expand_dims(np.mean(p_yx, 0), 0)  
    kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))  
    sum_kl_d = np.sum(kl_d, 1) 
    avg_kl_d = np.mean(sum_kl_d)  
    is_score = np.exp(avg_kl_d)  
    return is_score

def calculate_fid(mu1, sigma1, mu2, sigma2):
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

real_images = load_images_from_folder('Cloud', preprocess)
generated_images = load_images_from_folder('Cloud', preprocess)

real_probs = get_activations(real_images, inception)
generated_probs = get_activations(generated_images, inception)

# Inception Score
is_score = calculate_inception_score(generated_probs)
print(f'Inception Score: {is_score}')

# Frechet Inception Distance
mu_real, sigma_real = real_probs.mean(axis=0), np.cov(real_probs, rowvar=False)
mu_generated, sigma_generated = generated_probs.mean(axis=0), np.cov(generated_probs, rowvar=False)
fid = calculate_fid(mu_real, sigma_real, mu_generated, sigma_generated)
print(f'Frechet Inception Distance: {fid}')

Inception Score: 2.6062247520215522
Frechet Inception Distance: 4.373965035617817e-11


In [2]:
import os
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms
from torchvision.models import Inception_V3_Weights
from scipy.linalg import sqrtm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inception v3 
inception = models.inception_v3(weights=Inception_V3_Weights.IMAGENET1K_V1)
inception = inception.to(device).eval()
inception.fc = torch.nn.Sequential(inception.fc, torch.nn.Softmax(dim=1))

preprocess = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def load_images_from_folder(folder, preprocess):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        with Image.open(img_path) as img:
            img = preprocess(img)
            images.append(img)
    return torch.stack(images)

def get_activations(images, model, batch_size=64):
    model.eval()
    n_batches = len(images) // batch_size
    n_used_imgs = n_batches * batch_size
    pred_arr = np.empty((n_used_imgs, 1000))
    
    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size
        batch = images[start:end]
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch)    
        pred_arr[start:end] = pred.cpu().data.numpy()
    return pred_arr

def calculate_inception_score(p_yx, eps=1E-16):
    p_y = np.expand_dims(np.mean(p_yx, 0), 0)  
    kl_d = p_yx * (np.log(p_yx + eps) - np.log(p_y + eps))  
    sum_kl_d = np.sum(kl_d, 1)  
    avg_kl_d = np.mean(sum_kl_d)  
    is_score = np.exp(avg_kl_d) 
    return is_score

def calculate_fid(mu1, sigma1, mu2, sigma2):
    ssdiff = np.sum((mu1 - mu2)**2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid

real_images = load_images_from_folder('Cloud', preprocess)
generated_images = load_images_from_folder('result', preprocess)

real_probs = get_activations(real_images, inception)
generated_probs = get_activations(generated_images, inception)

# Inception Score
is_score = calculate_inception_score(generated_probs)
print(f'Inception Score: {is_score}')

# Frechet Inception Distance:
mu_real, sigma_real = real_probs.mean(axis=0), np.cov(real_probs, rowvar=False)
mu_generated, sigma_generated = generated_probs.mean(axis=0), np.cov(generated_probs, rowvar=False)
fid = calculate_fid(mu_real, sigma_real, mu_generated, sigma_generated)
print(f'Frechet Inception Distance: {fid}')

Inception Score: 1.8212098539603365
Frechet Inception Distance: 0.09150639215606834


From the FID perspective, the generation quality is quite good, and indeed, the generated images do appear more realistic compared to those produced by GAN models. The IS value is a bit low, but considering that both training and generation are focused on a single category of clouds, a lower IS value is also expected.